In [326]:
from icecream import ic

In [327]:
import pandas as pd
import numpy as np

Import energy consumption and occupation data, they are not of the same building but for now we'll use this data

In [328]:
energy_df = pd.read_csv("data/kWhperh_Humanitats.csv")
occupation_df = pd.read_csv("data/Ocupacio_enginyeria.csv")

In [329]:
occupation_df

,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions
0,01/02/2023,08:00,01/02/2023,17:00,9,QC/0011,NaN,600-12 instal�laci� videowall,0,,
1,01/02/2023,09:00,01/02/2023,12:00,3,Q3/1007,NaN,958-03 102773 Fonaments de la tecnologia de la...,0,,
2,01/02/2023,09:00,01/02/2023,12:00,3,Q3/1003,NaN,958-03 102773 Fonaments de tecnologia de la in...,0,,
3,01/02/2023,09:00,01/02/2023,15:00,6,Q4/0011,NaN,951-03 102401 Ampliaci� d'operacions de separaci�,0,,
4,01/02/2023,10:30,01/02/2023,13:00,"2,5",Q4/1005,NaN,1497-20 Revisi� d'ex�mens,0,,
...,...,...,...,...,...,...,...,...,...,...,...
2976,28/02/2023,18:00,28/02/2023,20:00,2,Q3/0003,10088 Enginyeria Inform�tica i Enginyeria de S...,102685 Arquitectura de Computadors i Perif�ric...,93,,
2977,,,,,,Q3/0003,10063 Enginyeria Electr�nica de Telecomunicaci�,102685 Arquitectura de Computadors i Perif�ric...,93,,
2978,,,,,,Q3/0003,10089 Enginyeria Inform�tica i Enginyeria Elec...,102685 Arquitectura de Computadors i Perif�ric...,93,,
2979,,,,,,Q3/0003,10100 Enginyeria Electr�nica de Telecomunicaci...,102685 Arquitectura de Computadors i Perif�ric...,93,,


In [330]:
energy_df

,Unnamed: 0,Date,Hour,L-Bibl.Humanitats (General) [kWh]
0,0,01/01/2022,00:00,"5,35"
1,1,01/01/2022,01:00,"4,64"
2,2,01/01/2022,02:00,"4,87"
3,3,01/01/2022,03:00,"5,43"
4,4,01/01/2022,04:00,"4,81"
...,...,...,...,...
8011,8011,30/11/2022,19:00,"80,70"
8012,8012,30/11/2022,20:00,"37,12"
8013,8013,30/11/2022,21:00,"24,40"
8014,8014,30/11/2022,22:00,"6,38"


the year and month doesn't match occupation so we'll have to invent a bit

In [331]:
for i,r in energy_df.iterrows():
    date = energy_df.loc[i]["Date"].split("/")
    energy_df.loc[i,"Date"] = date[0] + "/" + date[1] + "/2023"

energy_df

,Unnamed: 0,Date,Hour,L-Bibl.Humanitats (General) [kWh]
0,0,01/01/2023,00:00,"5,35"
1,1,01/01/2023,01:00,"4,64"
2,2,01/01/2023,02:00,"4,87"
3,3,01/01/2023,03:00,"5,43"
4,4,01/01/2023,04:00,"4,81"
...,...,...,...,...
8011,8011,30/11/2023,19:00,"80,70"
8012,8012,30/11/2023,20:00,"37,12"
8013,8013,30/11/2023,21:00,"24,40"
8014,8014,30/11/2023,22:00,"6,38"


# We want to start filtering by day

In [332]:
days = np.unique(occupation_df["Data inicial"])
days = days[1:] #eliminate " " date
days

array(['01/02/2023', '02/02/2023', '03/02/2023', '04/02/2023',
       '06/02/2023', '07/02/2023', '08/02/2023', '09/02/2023',
       '10/02/2023', '13/02/2023', '14/02/2023', '15/02/2023',
       '16/02/2023', '17/02/2023', '20/02/2023', '21/02/2023',
       '22/02/2023', '23/02/2023', '24/02/2023', '27/02/2023',
       '28/02/2023'], dtype=object)

But... Is there any activity that data inicial is different to data final? (last many days?)

In [333]:
occupation_df[occupation_df["Data inicial"] != occupation_df["Data final"]]

,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions


ok, there are none... everything lasts one day

# At each hour, each day, what classes are being taken and where?

In [334]:
#to compare dates and hours
from datetime import datetime

In [335]:
#convert energy df into datetime object
for i, row in energy_df.iterrows():
    energy_df.loc[i, "Hour"] = datetime.strptime(energy_df.loc[i, "Hour"], "%H:%M").time()

In [336]:
#energy has them all so we generate from here
hours_in_day = np.unique(energy_df[energy_df["Date"] == "01/02/2023"]["Hour"])
hours_in_day

array([datetime.time(0, 0), datetime.time(1, 0), datetime.time(2, 0),
       datetime.time(3, 0), datetime.time(4, 0), datetime.time(5, 0),
       datetime.time(6, 0), datetime.time(7, 0), datetime.time(8, 0),
       datetime.time(9, 0), datetime.time(10, 0), datetime.time(11, 0),
       datetime.time(12, 0), datetime.time(13, 0), datetime.time(14, 0),
       datetime.time(15, 0), datetime.time(16, 0), datetime.time(17, 0),
       datetime.time(18, 0), datetime.time(19, 0), datetime.time(20, 0),
       datetime.time(21, 0), datetime.time(22, 0), datetime.time(23, 0)],
      dtype=object)

Remove entries with nan values in occupation df. No date or no hour 

In [337]:
occupation_df[occupation_df["Data inicial"] == " "]

,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions
80,,,,,,Q3/0013,10088 Enginyeria Inform�tica i Enginyeria de S...,102689 Components i Circuits Electr�nics - 311,Sense grup de matricula,,
81,,,,,,Q3/0013,10063 Enginyeria Electr�nica de Telecomunicaci�,102689 Components i Circuits Electr�nics - 311,Sense grup de matricula,,
82,,,,,,Q3/0013,10064 Enginyeria de Sistemes de Telecomunicaci�,102689 Components i Circuits Electr�nics - 311,Sense grup de matricula,,
83,,,,,,Q3/0013,10100 Enginyeria Electr�nica de Telecomunicaci...,102689 Components i Circuits Electr�nics - 311,Sense grup de matricula,,
84,,,,,,Q3/0013,10089 Enginyeria Inform�tica i Enginyeria Elec...,102689 Components i Circuits Electr�nics - 311,Sense grup de matricula,,
...,...,...,...,...,...,...,...,...,...,...,...
2974,,,,,,Q3/0009,10089 Enginyeria Inform�tica i Enginyeria Elec...,102769 Informaci� i Seguretat - 453,Sense grup de matricula,,
2977,,,,,,Q3/0003,10063 Enginyeria Electr�nica de Telecomunicaci�,102685 Arquitectura de Computadors i Perif�ric...,93,,
2978,,,,,,Q3/0003,10089 Enginyeria Inform�tica i Enginyeria Elec...,102685 Arquitectura de Computadors i Perif�ric...,93,,
2979,,,,,,Q3/0003,10100 Enginyeria Electr�nica de Telecomunicaci...,102685 Arquitectura de Computadors i Perif�ric...,93,,


In [338]:
#we'll remove those
occupation_df = occupation_df[occupation_df["Data inicial"] != " "]

In [339]:
occupation_df[occupation_df["Hora inicial"] == " "]

,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions


Wait, but first we should convert all hours into datetime object so we can easely compare them.

In [340]:
for i, row in occupation_df.iterrows():
    occupation_df.loc[i]["Hora inicial"] = datetime.strptime(occupation_df.loc[i]["Hora inicial"] ,"%H:%M").time()
    occupation_df.loc[i]["Hora final"] = datetime.strptime(occupation_df.loc[i]["Hora final"] ,"%H:%M").time()

In [341]:
occupation_df

,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions
0,01/02/2023,08:00:00,01/02/2023,17:00:00,9,QC/0011,NaN,600-12 instal�laci� videowall,0,,
1,01/02/2023,09:00:00,01/02/2023,12:00:00,3,Q3/1007,NaN,958-03 102773 Fonaments de la tecnologia de la...,0,,
2,01/02/2023,09:00:00,01/02/2023,12:00:00,3,Q3/1003,NaN,958-03 102773 Fonaments de tecnologia de la in...,0,,
3,01/02/2023,09:00:00,01/02/2023,15:00:00,6,Q4/0011,NaN,951-03 102401 Ampliaci� d'operacions de separaci�,0,,
4,01/02/2023,10:30:00,01/02/2023,13:00:00,"2,5",Q4/1005,NaN,1497-20 Revisi� d'ex�mens,0,,
...,...,...,...,...,...,...,...,...,...,...,...
2968,28/02/2023,18:00:00,28/02/2023,19:00:00,1,Q4/1009,10062 Enginyeria Inform�tica,102768 Intel�lig�ncia Artificial - 451,Sense grup de matricula,,
2971,28/02/2023,18:00:00,28/02/2023,19:00:00,1,Q4/1003,10130 Intel�lig�ncia Artificial / Artificial I...,106573 Xarxes Neuronals i Aprenentatge Profund...,Sense grup de matricula,,
2972,28/02/2023,18:00:00,28/02/2023,19:00:00,1,Q3/0009,10088 Enginyeria Inform�tica i Enginyeria de S...,102769 Informaci� i Seguretat - 453,Sense grup de matricula,,
2975,28/02/2023,18:00:00,28/02/2023,20:00:00,2,Q2/1009,10105 Gesti� de Ciutats Intel�ligents i Sosten...,104539 Seguretat i Privacitat Dels Sistemes d'...,52,,


The following loop iterates all days and all hours and for each (day-hour) slices classes bein taken at that time

In [342]:
for d in days:
    #filter dataset to see activities that day
    day2day_ocu = occupation_df[occupation_df["Data inicial"] == d ] 
    for h in hours_in_day:
        hour2hour_ocu = day2day_ocu[(day2day_ocu["Hora inicial"] <= h) & (day2day_ocu["Hora final"] > h)]

To analyze the impact of classes taken to consumtion:

# 1. Assume concurrent classes consume the same

We'll give each class at each hour a mean KW (power) draw

In [343]:
occupation_df["KW"] = np.zeros(len(occupation_df))
occupation_df

/var/folders/j8/64lnvrmj50q5dv_5dj_ztbz40000gn/T/ipykernel_10075/3208925040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  occupation_df["KW"] = np.zeros(len(occupation_df))


,Data inicial,Hora inicial,Data final,Hora final,Total hores,Espai,Estudi,Activitat,Alumnes matriculats,Modalitat docencia,Observacions,KW
0,01/02/2023,08:00:00,01/02/2023,17:00:00,9,QC/0011,NaN,600-12 instal�laci� videowall,0,,,0.0
1,01/02/2023,09:00:00,01/02/2023,12:00:00,3,Q3/1007,NaN,958-03 102773 Fonaments de la tecnologia de la...,0,,,0.0
2,01/02/2023,09:00:00,01/02/2023,12:00:00,3,Q3/1003,NaN,958-03 102773 Fonaments de tecnologia de la in...,0,,,0.0
3,01/02/2023,09:00:00,01/02/2023,15:00:00,6,Q4/0011,NaN,951-03 102401 Ampliaci� d'operacions de separaci�,0,,,0.0
4,01/02/2023,10:30:00,01/02/2023,13:00:00,"2,5",Q4/1005,NaN,1497-20 Revisi� d'ex�mens,0,,,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2968,28/02/2023,18:00:00,28/02/2023,19:00:00,1,Q4/1009,10062 Enginyeria Inform�tica,102768 Intel�lig�ncia Artificial - 451,Sense grup de matricula,,,0.0
2971,28/02/2023,18:00:00,28/02/2023,19:00:00,1,Q4/1003,10130 Intel�lig�ncia Artificial / Artificial I...,106573 Xarxes Neuronals i Aprenentatge Profund...,Sense grup de matricula,,,0.0
2972,28/02/2023,18:00:00,28/02/2023,19:00:00,1,Q3/0009,10088 Enginyeria Inform�tica i Enginyeria de S...,102769 Informaci� i Seguretat - 453,Sense grup de matricula,,,0.0
2975,28/02/2023,18:00:00,28/02/2023,20:00:00,2,Q2/1009,10105 Gesti� de Ciutats Intel�ligents i Sosten...,104539 Seguretat i Privacitat Dels Sistemes d'...,52,,,0.0


In [347]:
#first sum up 
for d in days:
    #filter dataset to see activities that day
    day2day_ocu = occupation_df[occupation_df["Data inicial"] == d ] 
    for h in hours_in_day:
        hour2hour_ocu = day2day_ocu[(day2day_ocu["Hora inicial"] <= h) & (day2day_ocu["Hora final"] > h)]
        energy = energy_df[(energy_df["Date"] == d) & (energy_df["Hour"] == h)]["L-Bibl.Humanitats (General) [kWh]"].values[0]
        energy = float(energy.replace(",","."))
        #get every hour kwh/class and add to the consuption of each instance
        occupation_df.loc[hour2hour_ocu.index,"KW"] = (occupation_df.loc[hour2hour_ocu.index,"KW"] + energy)/len(hour2hour_ocu)
        #ic(energy)
        #ic(occupation_df.loc[hour2hour_ocu.index,"KW"])

By subject "activity" description

In [353]:
occupation_df.sort_values(by=['KW'])[-100:]["Activitat"].value_counts()

115-19 JPO 2023 Xerrada General                                     12
102685 Arquitectura de Computadors i Perif�rics - 33                 5
115-18 Ritsi Laia vilardell                                          5
115-18 Curs Memenginy (Cristina Fernandez)                           4
1268-20 TAV (MEUBA)                                                  4
1268-09 Doc�ncia M�ster EBA 1r curs 2n semestre                      4
104539 Seguretat i Privacitat Dels Sistemes d'Informaci� - 61        3
102743 Enginyeria del Software - 451                                 2
115-19 JPO 2023 GED                                                  2
104540 Ci�ncia de Dades - 61                                         2
958-03 102742 Tecnol.desenv. internet i web (examen PLAB)            2
104536 Sistemes d'Informaci� Geogr�fica - 61                         2
1268- 20 BS (MUEBA)                                                  2
104550 Models d'Innovaci� Urbana i Ci�ncia Ciutadana - 611           2
958-03

occupation_df.sort_values(by=['KW'])[-100:]["Activitat"].value_counts()

In [354]:
occupation_df.sort_values(by=['KW'])[-100:]["Espai"].value_counts()

Q1/1003      7
Q2/1009      7
Q4/1005      7
Q4/1003      6
QC/0011      6
Q3/0013      6
Q3/0003      5
Q3/0009      4
QC/0019      4
Q1/0003      4
Q4/1013      4
QC/3115.2    4
Q2/1005      3
QC/0029      3
QC/3115.1    3
Q3/1003      3
Q1/1011      3
Q3/1007      3
QC/0015      3
Q1/0019      3
Q1/1007      2
Q3/1011      2
Q4/0003      1
Q4/0011      1
Q4/0007      1
QC/3115.3    1
Q2/0007      1
QC/3115.4    1
Q1/0007      1
QC/3115.5    1
Name: Espai, dtype: int64

It might indicate a correlation but not a causality.

This should probably be contrasted with how much hours does an activity do and how many hours is each class used. Outliers should then be analyzed. Unfrequent activities/rooms associated to high consumtion.